In [290]:
import re
subitem_match = re.compile(r"^(?:([#]+)\s*)").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match
header_match = re.compile(r"^(?:([\*\+]*) )?([\+\*]+)([^\*\+]+?)\:?([\+\*]+)\:?\s*(.+)?$").match
frs_match = re.compile(r"(FRS\d+)\.?([\d\.]+)?").match
num_escape_match = re.compile(r"^(\*{2})(\#+)").match

class FRSParser():
    VALID_INDENTS = "*+"
    
    # inputs
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    LINE_HEADER = 3
    LINE_NUM_ESCAPE = 4
    
    # states
    ST_INIT = 0
    ST_NONUM = 1
    ST_FRS = 2
    
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = []
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
        self.sm_st = self.ST_INIT
        self.sm = [
            # ST_INIT             ST_NONUM             ST_FRS
            [self.set_frs,        self.set_frs,        self.set_frs],        # LINE_FRS
            [None,                None,                self.set_subitem],    # LINE_SUBITEM
            [None,                None,                None],                # LINE_OTHER
            [self.set_header,     self.set_header,     self.set_header],     # LINE_HEADER
            [self.set_num_escape, self.set_num_escape, self.set_num_escape], # LINE_NUM_ESCAPE
        ]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
        
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
    
    def format_lvl(self, lvl):
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def feed(self, line):
        typ, arg = self._feed(line)
        func = self.sm[typ][self.sm_st]
        if func:
            st, rv = func(arg)
            self.sm_st = st
        else:
            rv = line
        return rv
    
    def _feed(self, line): 
        m = header_match(line)
        if m:
            ind, fmt, s, fmt2, _ = m.groups()

            # Verify format to avoid false positive. 
            if fmt == fmt2[::-1]:
                m2 = frs_match(s)
                if m2:
                    typ = self.LINE_FRS
                    frs, nums = m2.groups()
                    arg = (line, ind, fmt, frs, nums)
                else:
                    typ = self.LINE_HEADER
                    arg = (line, ind, fmt, s)
                return typ, arg
        m = subitem_match(line)
        if m:
            typ = self.LINE_SUBITEM
            arg = line, len(m.group(1) or "")
            return typ, arg
        m = num_escape_match(line)
        if m:
            typ = self.LINE_NUM_ESCAPE
            arg = line,
            return typ, arg
        return self.LINE_OTHER, (line,)
    
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def parse_text(self, text):
        for line in text.splitlines():
            yield self.feed(line)
    
    # State machine methods
                                
    def set_frs(self, arg):
        line, indent, fmt, frs, nums = arg
        self.ilvl = len(indent or "")
        self.fmt = fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        return self.ST_FRS, line
    
    def set_header(self, arg):
        return self.ST_NONUM, arg[0]
    
    def set_subitem(self, arg):
        line, n = arg
        self.set_indent(n)
        fmt = self.frs_string()
        try:
            _, tail = line.split(" ", 1)
        except ValueError:
            v = fmt
        else:
            v = " ".join((fmt, tail))
        return self.sm_st, v
    
    def set_num_escape(self, arg):
        line, = arg
        return self.sm_st, line.lstrip("*")
    
NumState = FRSParser

In [302]:
ntests = 0
passed = 0
errors = []
def assert_ab(a, b):
    global ntests, passed, errors
    ntests += 1
    try:
        assert a == b, "%r != %r" % (a, b)
    except AssertionError as e:
        val = str(e)
        if isinstance(a, str) and isinstance(b, str):
            val += "\n" + str(a.splitlines()) + "\n" + str(b.splitlines())
        errors.append(val)
    else:
        passed += 1

def assert_header_match(s, exp):
    assert_ab(header_match(s).groups(), exp)
    
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("*URS*", (None, "*", "URS", "*", None))
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*", ("*", "*", "FRS1234", "*", None))
assert_header_match("*FRS1234.5.6*", (None, "*", "FRS1234.5.6", "*", None))
assert_header_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234.5.6", "+*", None))
assert_header_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234.5.6.7.8", "+*", None))
assert_header_match("* *FRS1234:*", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*:", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", "*", "Bob"))
assert header_match("**") == None
assert header_match("***") == None

def assert_num_escape(s, exp):
    m = num_escape_match(s)
    r = m.groups() if m else None
    assert_ab(r, exp)
    
assert_num_escape("**# Bob", ("**", "#"))
assert_num_escape("**## Bob", ("**", "##"))
assert_num_escape("**### Bob", ("**", "###"))
assert_num_escape("# Bob", None)
assert_num_escape("## Bob", None)
assert_num_escape("### Bob", None)
assert_num_escape("* Bob", None)
assert_num_escape("** Bob", None)
assert_num_escape("*** Bob", None)

def set_str(st, s):
    ind, fmt, fs, _, _ = header_match(s).groups()
    frs, nums = frs_match2(fs).groups()
    st.set_frs((s, ind, fmt, frs, nums))
    

def test_state(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1*", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1*", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    set_str(st, s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    set_str(st, s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

def assert_feed_indent(s, feed, exp):
    st = NumState()
    set_str(st, s)
    st.sm_st = st.ST_FRS
    st.feed(feed)
    assert_ab(st.frs_string(), exp)
    
assert_feed_indent("*FRS1234*", "# Bob", "* *FRS1234.1:*")
assert_feed_indent("*FRS1234*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "# Bob", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## Bob", "** *FRS1234.1.1.1:*")
assert_feed_indent("* *FRS1234.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("* *FRS1234*", "## Bob", "** *FRS1234.1:*")
assert_feed_indent("** *FRS1234.1*", "## Bob", "** *FRS1234.2:*")
assert_feed_indent("* *FRS1234.1*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("** *FRS1234.1.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("*FRS1234.1*", "# ", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## ", "** *FRS1234.1.1.1:*")
assert_feed_indent("*FRS1234.1*", "#", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "##", "** *FRS1234.1.1.1:*")

def assert_double_parse(t, d=5):
    r = list(NumState(d).parse_text(t))
    r2 = list(NumState(d).parse_text("\n".join(r)))
    assert_ab(r, r2)
    
dpt = """*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""
assert_double_parse(dpt)

def assert_new_state(s, exp, nxt=1, ind=0):
    st = NumState()
    for l in s.splitlines():
        st.feed(l)
    for _ in range(nxt):
        st.next()
    for _ in range(ind):
        st.indent()
    assert_ab(st.frs_string(), exp)
    
assert_new_state("*FRS123.1*\n*FRS31.2*", "*FRS31.3:*")
assert_new_state("*FRS123.1*\n*FRS123.3*", "*FRS123.4:*")

def assert_parse_text(txt, exp):
    st = NumState()
    lines = list(st.parse_text(txt))
    res = "\n".join(lines)
    assert_ab(res, exp)

lines = """*FRS1234.1:*
#"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*"""
assert_parse_text(lines, exp)

lines="""*FRS1234.1:*
#
## Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
#
##
"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*
** *FRS1234.1.1.1:* Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
* *FRS1234.2.1:*
** *FRS1234.2.1.1:*"""

assert_parse_text(lines, exp)

lines = "+*FRS*+\n*FRS1234:*\n#"
exp = "+*FRS*+\n*FRS1234:*\n* *FRS1234.1:*"
assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
*** Bob
## Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
*** Bob
** *FRS2908.1.1.1:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
#### Biff
*** Bob
# Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
**** *FRS2908.1.1.1.1.1:* Biff
*** Bob
* *FRS2908.1.2:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.0*
# Bob1
## Bob2
*** Fred1
*** Fred2
## Bob3
## Bob4
## Bob5
*** Fred3
#### Bob6
#### Bob7
#### Bob8
*** Fred4
# Bob9
"""

exp = """*FRS2908.0*
* *FRS2908.0.1:* Bob1
** *FRS2908.0.1.1:* Bob2
*** Fred1
*** Fred2
** *FRS2908.0.1.2:* Bob3
** *FRS2908.0.1.3:* Bob4
** *FRS2908.0.1.4:* Bob5
*** Fred3
**** *FRS2908.0.1.4.1.1:* Bob6
**** *FRS2908.0.1.4.1.2:* Bob7
**** *FRS2908.0.1.4.1.3:* Bob8
*** Fred4
* *FRS2908.0.2:* Bob9"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** Fred:
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** Fred:
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** considered +/- 15 degrees
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** considered +/- 15 degrees
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

def assert_parse_empty(s, exp):
    st = NumState()
    res = None
    for res in st.parse_text(s):
        pass
    assert_ab(res, exp)
    
assert_parse_empty("*User Requirements*\n# Bar", "# Bar")

def finish():
    print("%d/%d tests passed successfully" % (passed, ntests))
    for e in errors:
        print(e)
finish()

93/93 tests passed successfully


In [317]:
lines=r"""
*+FRS3226.01+*: User facing names for DO globals shall indicate whether they are used for Air or Mag, according to the following table:
* Note that the contents of this table will be refered to as "Air specific globals" and "Mag specific globals" throughout this FRS Item. 
!global_names.png!


*+Implementation of FRS3226.01+*
PBS Global.vi:
# Change the cluster named "DOO2Control" to "DOO2ControlAir"
# Duplicate the new "DOO2ControlAir" cluster, re-name it to "DOO2ControlMag", and re-name the "DOO2ControlMag.PGain(mLPM/%)" to "DOO2ControlMag.PGain(%/%)"
# Duplicate the "DOO2RangeAuto(mLPM)" cluster and re-name the copy "DOO2RangeAuto(%)"
# Duplicate "DOO2RangeManMax(mLPM)" and re-name the copy "DOO2RangeManMax(%)"
# Duplicate "DOO2FlowControllerRequest(mLPM)" and re-name the copy "DOO2FlowControllerRequest(%)"
# Duplicate "DOO2FlowControllerRequestLimited(mLPM)" and re-name the copy "DOO2FlowControllerRequestLimited(%)"
# Duplicate "DOO2FlowUser(mLPM)" and re-name the copy "DOO2FlowUser(%)"

*+FRS3226.02+*
# The bioreactor shall write to and read from Mag-specific globals when configured as a Mag Drive
# The bioreactor shall write to and read from Air-specific globals when configured as a Air Drive

*Implementation of +FRS3226.02+*
DO Module.vi:
# If the model is Mag, make the following changes to the globals used:
** DOO2FlowUser(%) instead of DOO2FlowUser(mLPM)
** DOO2RangeManMax(%) instead of DOO2RangeManMax(mLPM)
** DOO2RangeAuto(%) instead of DOO2RangeAuto(mLPM)
** DOO2ControlMag instead of DOO2ControlAir
# If the model is Mag, use "Coerce for Percent Inputs.vi" for the output from the O2 PID, rather than "Coerce to Positive Number.vi"
# If the model is Mag, use DOO2FlowControllerRequest(%) instead of DOO2FlowControllerRequest(mLPM)

*+FRS3226.03+*
New VI, Calc O2 Limited.vi:
# This will be called in the Calc Actual N2 and CO2 Module.vi.
# It will calculate both DOO2FlowControllerRequestLimited(mLPM) and DOO2FlowControllerRequestLimited(%), for all model types.
!Calc O2 Limited Mag.png!
!Calc O2 Limited Air.png!

*+FRS3226.04+*
Calc Actual N2 and CO2 Module.vi:
# This is far simpler than before. It calls the new Calc O2 Limited VI, and also calculates the pHCO2ActualRequest and DON2FlowActualRequest in the same way, via a for loop and shift register.
!Calc Actual N2 and CO2 Module zero.png!
!Calc Actual N2 and CO2 Module nonzero.png!

*+FRS3226.05+*
Calc Gas Flows Module.vi:
# If the model is Mag, calculate the "MainGasFeedback(LPM) in the "Calc Flows" case as the sum of all 4 gases
!Calc Gas Flows Module Calc Flows Mag.png!
!Calc Gas Flows Module Calc Flows Air.png!
# In the "Calc Voltages" case, replace the instance of "Calc Gasses with PWM for O2.vi" with an instance of "Calc Gasses for PWM.vi"
** Keep the Ticks/cycle, Max gas period, Gas On Time, and Gas Constants inputs the same.
** For the Gas Out (LPM) input, feed the DOO2FlowControllerRequestLimited(mLPM) global, multiplied by 0.001
** There is no longer any need to unbundle the O2 raw voltage, nor to call the O2 Min Volume (L) global, nor to write to the DOO2FlowControllerRequestLimited(mLPM).
** The Calc Gasses with PWM for O2.vi shall be deleted from the project
# If the model is an Air drive, the Gas Out (L) output from the O2 MFC's instance of Calc Gasses for PWM.vi should be added to the AgMainGasActualRequest(LPM), and then fed downstream into everything else.  This allows the CO2 and N2 percents to be converted to flow rates using the correct total gas flow for both Air and Mag.  And then, when calculating Air flow, the Air drive is main gas - (N2 + CO2), and for Mag drive it's main gas - (N2 + CO2 + O2).
!Calc Gas Flows Module Calc Voltages Mag.png!
!Calc Gas Flows Module Calc Voltages Air.png!

*+FRS3226.07+*
Get Variables to Send to Atom.vi:
# If the model is Mag, make the following changes to the globals used:
** DOO2FlowControllerRequestLimited(%) instead of DOO2FlowControllerRequestLimited(mLPM)
** DOO2FlowUser(%) instead of DOO2FlowUser(mLPM)
** DOO2RangeManMax(%) instead of DOO2RangeManMax(mLPM)

*+FRS3226.08+*
NetworkFetchOnRIO.vi:
# If the model is Mag, make the following changes to the globals used:
** DOO2FlowUser(%) instead of DOO2FlowUser(mLPM)

*+FRS3226.09+*
Get Static Info.vi
# If the model is Mag, make the do manUpUnit %, if it is Air make the do manUpUnit mL/min.
# If the model is Mag, make the do manUpDecimals 1.  If it is Air make the do manUpDecimals 0.

*+FRS3226.10+*
Set mode or sp.vi
# For the "do" sensor, "Man" case, if the model is Mag, make the Format String use units of percent, the way it does for N2, rather than mLPM

*+FRS3226.11+*
System Variables
# Change the System Variables Mag typedef so in the DO group the following settings have new names:
** O2 P Gain (mLPM/DO%) becomes O2 P Gain (%/DO%)
** O2 Manual Max (mLPM) becomes O2 Manual Max (%)
** O2 Auto Max (mLPM) becomes O2 Auto Max (%)
# Retrieve System Variables from file.vi:
## For the Air case, it needs to write to:
*** The DOO2ControlAir cluster
*** The DOO2RangeManMax(mLPM)
*** The DOO2RangeAuto(mLPM) cluster
## For the Mag case, it needs to write to:
*** The DOO2ControlMag cluster
*** The DOO2RangeManMax(%)
*** The DOO2RangeAuto(%) cluster
# Write Shared Sys Vars for Air.vi and  Write Shared Sys Vars for Mag.vi
** The DO case should no longer unbundle the following (and thus the corresponding globals also shouldn't be written to):
*** O2 P Gain
*** O2 I Time
*** O2 D Time
*** O2 Manual Max
*** O2 Auto Max
"""
res = "\n".join(NumState(3).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)

Map Network Drive
!Map_Drive.png!
Numbered Mockup:
!Map_Drive_Numbered.png!

*FRS3010.1* Appearance
# Item 1 is a selectable listbox labeled Available Drives
# Item 2 is a selectable lisbotx labeled Network Drives
# Item 3 is a string control labeled Map Network Drive [\\computerName\foldername]
# Item 4 is a Boolean button labeled Delete
# Item 5 is a Boolean button labeled Map
# Item 6 is a Boolean button labeled Close

*FRS3010.2* Functionality
* *FRS3010.2.1:* At all times:
** *FRS3010.2.1.1:* Item 1 shall be populated with a list of letters from A-Z representing available drive letters.
** *FRS3010.2.1.2:* Item 2 shall be populated with a list of all network drives currently mapped. 
** *FRS3010.2.1.3:* There shall always be exactly 1 item selected in items 1 and 2, unless the lists are empty.
** *FRS3010.2.1.4:* When a user clicks inside Items 1 or 2, the item in the list (if any) shall be selected.
** *FRS3010.2.1.5:* When Item 3 is selected, a keyboard shall appear per FRS3288 

In [320]:
lines = r"""*+URS+*
See 1.5 in issue #3194

*+Specification+*
*FRS989.0*
# Login attempts to the Shell or the Web UI (i.e. the 'login' call) will have behavior illustrated in the following flow chart:
!Login.png!
## A nonexistent user shall also to be considered to "not have permissions to log in to UI"
## If a user's group has '0' as the Password Expiration Period (days), the password shall never expire.
## Failure to send the email on a failed login attempt shall not impact response time to the Web UI to a noticable degree. 
# The login Server call shall be formatted as follows:
## The response shall be formatted as JSON with the following structure:
<pre><code class="json">
{"Result":true,"Message":"Message to user"}
</code></pre>
## The Result field shall return "True" for successful logins, and otherwise "False". 
## The Message field shall contain a message to be displayed to the user, as defined in FRS989.0.1
## If no message is specified, the message field shall contain "True".
### This is to contain the message to the UI, as defined in the flow chart
### If no message is defined in the flow chart, send True
# Not included in the flow chart is assigning sessions.  A session shall be assigned after writing a user event to the DB.

Using procedure:
# Require all admin password to be at least 8 characters long and all user password to be at least 5 characters long.
# Require each username to be associated with an actual physical employee"""
value = clipboard.paste()
value = lines
res = "\n".join(NumState(3).parse_text(value))
print(res)
import clipboard
clipboard.copy(res)

*+URS+*
See 1.5 in issue #3194

*+Specification+*
*FRS989.0*
* *FRS989.0.1:* Login attempts to the Shell or the Web UI (i.e. the 'login' call) will have behavior illustrated in the following flow chart:
!Login.png!
** *FRS989.0.1.1:* A nonexistent user is also to be considered to "not have permissions to log in to UI"
** *FRS989.0.1.2:* If a user's group has '0' as the Password Expiration Period (days), the password will never expire.
** *FRS989.0.1.3:* For a failed login attempt from the Web UI, the email should happen in parallel - if there's a problem sending the email, it shouldn't affect the Server's response time to the Web UI.
* *FRS989.0.2:* The login Server call will be in JSON format as follows
<pre><code class="json">
{"Result":true,"Message":"Message to user"}
</code></pre>
* *FRS989.0.3:* For login attempts in the Web UI, sent the JSON response for the Login call as follows:
** *FRS989.0.3.1:* Result
*** *FRS989.0.3.1.1:* If the user successfully logged in, send True
*** *